Packages to install:

    brew install tcl-tk ghostscript
    pip install camelot-py[cv]
    pip install PyPDF2
    

In [140]:
import camelot
import PyPDF2
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import os

# Functions

In [2]:
def fetch_table(filename):
    soup = BeautifulSoup(open(filename), "html.parser")
    return soup.findAll('table')[0]

In [3]:
def get_season_raw_data(target_table):
    data_list = []
    for row in target_table.find_all('tr'):
        row_list = []
        for item in row.find_all('th'):
            row_list.append(item.text)
        for item in row.find_all('td'):
            row_list.append(item.text)
            try:
                row_list.append(item.a['href'])
            except:
                continue
        if len(row_list) == 8 and row_list[-1].replace(' ','') == 'Name':
            data_list.append(row_list + ['URL'])
        if len(row_list) == 9:
            data_list.append(row_list)
    return data_list

In [4]:
def download_pdf(year, date, url):
    f = requests.get(url)
    open(f'{str(year)}/{date}.pdf', 'wb').write(f.content)

In [14]:
def get_season_net_df(filename, downloaded=True):
    target_table = fetch_table(filename)
    data_list = get_season_raw_data(target_table)
    
    cols = [col.lower().strip(' ').replace(' ', '_') 
            for col in data_list[0]]
    df = pd.DataFrame(data_list[1:], columns=cols)
    if len(set(df['year'])) != 1:
        raise ValueError('Recheck the .html file')
    else:
        year = list(set(df['year']))[0]
    
    df = df.drop(['sport', 'division', 'gender', 'year', 'document_type'], axis=1)
    df['through_date'] = df['through_date']\
    .apply(lambda x: str(datetime.strptime(x, '%m/%d/%Y'))
           .split(' ')[0])
    
    if downloaded == False:
        print('Downloading pdfs...')
        for row in df[['through_date', 'url']].values:
            download_pdf(year, row[0], row[1])
        print('Complete Downloading.')
    
    df['filename'] = df['through_date'].apply(lambda x: f'{year}/{x}.pdf')
    df['page_number'] = df['filename'].apply(lambda x: PyPDF2.PdfFileReader(x).getNumPages())
    
    return df

In [6]:
def pdf_to_tables(pdf_filename, num_page):
    pages = ','.join([str(x) for x in list(range(1, 1 + num_page))])
    tables = camelot.read_pdf(pdf_filename, flavor='stream', pages=pages)
    return tables

In [142]:
def tables_to_df(tables, num_page, cols):
    df = pd.concat([pd.DataFrame([row for row in tables[i].df.values 
                                  if '' not in row or '[]' in row], columns=cols)
                    for i in range(num_page)])
    if df.shape[0] != 353:
        raise ValueError(f'There should be 353 teams, not {df.shape[0]}')
    else:
        if '' in df.columns:
            df = df.drop('', axis=1)
            df['net_rank'] = [str(i+1) for i in range(df.shape[0])]
        if 'sos' not in df.columns:
            df['sos'] = np.nan
            df['nc_sos'] = np.nan
            cols = ['team', 'net_rank', 'avg_opp_net', 'avg_opp_rank',
                    'record', 'conf_record', 
                    'non_conf_record', 'road_record', 'sos', 'nc_sos']  + \
                   [f'quadrant_{i}_record' for i in range(1, 5)]
            df = df[cols]
        return df

In [143]:
def get_net_df(pdf_filename, num_page, cols):
    tables = pdf_to_tables(pdf_filename, num_page)
    df = tables_to_df(tables, num_page, cols)
    df['filename'] = pdf_filename
    return df

In [70]:
def get_net_cols(pdf_filename):
    tables = camelot.read_pdf(pdf_filename, flavor='stream', pages='1')
    cols = [row for row in tables[0].df.values if 'Team' in row]
    if 'NET' not in cols[0]:
        print(f'No NET in this file: {pdf_filename}')
        return None
    else:
        return len(cols[0])

# Run

In [17]:
# net_df = get_season_net_df('NCAA RPI Archive - Home.htm', downloaded=False)
net_df = get_season_net_df('NCAA RPI Archive - Home.htm')
# net_df.to_csv('2019_NET.csv', index=False)

In [74]:
net_df.head()

,report_type,through_date,name,url,filename,page_number
0,Final,2019-04-08,Final 2019 MBB Nitty Gritty,https://extra.ncaa.org/solutions/rpi/Stats%20L...,2019/2019-04-08.pdf,13
1,Selection,2019-03-17,2019 Selections Nitty Gritty,https://extra.ncaa.org/solutions/rpi/Stats%20L...,2019/2019-03-17.pdf,13
2,Current,2019-03-16,"NET Nitty Gritty - Games through March 16, 2019",https://extra.ncaa.org/solutions/rpi/Stats%20L...,2019/2019-03-16.pdf,13
3,Current,2019-03-15,"NET Nitty Gritty - Games through March 15, 2019",https://extra.ncaa.org/solutions/rpi/Stats%20L...,2019/2019-03-15.pdf,13
4,Current,2019-03-14,"NET Nitty Gritty - Games through March 14, 2019",https://extra.ncaa.org/solutions/rpi/Stats%20L...,2019/2019-03-14.pdf,13


In [75]:
net_df['col_num'] = net_df.filename.apply(get_net_cols)

No NET in this file: 2019/2019-02-01.pdf
No NET in this file: 2019/2019-01-31.pdf


In [76]:
# Add a column of columns
col_dict = {}
col_dict[12] = ['team', 'net_rank', 'avg_opp_net', 'avg_opp_rank',
                'record', 'conf_record', 
                'non_conf_record', 'road_record'] + \
               [f'quadrant_{i}_record' for i in range(1, 5)]
col_dict[14] = ['team', 'net_rank', 'avg_opp_net', 'avg_opp_rank',
                'record', 'conf_record', 
                'non_conf_record', 'road_record', 'sos', 'nc_sos']  + \
               [f'quadrant_{i}_record' for i in range(1, 5)]
col_dict[15] = ['', 'team', 'net_rank', 'avg_opp_net', 'avg_opp_rank',
                'record', 'conf_record', 
                'non_conf_record', 'road_record', 'sos', 'nc_sos']  + \
               [f'quadrant_{i}_record' for i in range(1, 5)]
net_df['cols'] = net_df['col_num'].map(col_dict)

In [151]:
start = 1
for row in net_df[~net_df.col_num.isna()][['filename', 'page_number', 'cols']].values:
    csv_name = row[0].replace('.pdf', '.csv').replace('2019', '2019_clean')
    df = get_net_df(row[0], row[1], row[2])
    if df.shape != (353, 15):
        raise ValueError('Dataframe size should be 353 x 15.')
    df.to_csv(csv_name, index=False)
    if start % 10 == 0:
        print(start, 'pdfs completed.')
    start += 1

10 pdfs completed.
20 pdfs completed.
30 pdfs completed.
40 pdfs completed.
50 pdfs completed.
60 pdfs completed.
70 pdfs completed.
80 pdfs completed.
90 pdfs completed.


In [156]:
overall_net_df = pd.concat([pd.read_csv(fn.replace('.pdf', '.csv').replace('2019', '2019_clean'))
                            for fn in net_df[~net_df.col_num.isna()]['filename'].values])
overall_net_df.shape[0] == 353 * len(net_df[~net_df.col_num.isna()])

True

In [158]:
overall_net_df.head()

,team,net_rank,avg_opp_net,avg_opp_rank,record,conf_record,non_conf_record,road_record,sos,nc_sos,quadrant_1_record,quadrant_2_record,quadrant_3_record,quadrant_4_record,filename
0,Virginia,1,91,40,35-3,17-3,18-0,10-1,18,55,17-3,5-0,6-0,7-0,2019/2019-04-08.pdf
1,Gonzaga,2,129,82,33-4,17-1,16-3,9-1,40,33,6-4,6-0,10-0,11-0,2019/2019-04-08.pdf
2,Duke,3,79,13,32-6,18-4,14-2,7-2,NaN,6 8,13-5,6-1,6-0,7-0,2019/2019-04-08.pdf
3,Kentucky,4,82,17,30-7,16-5,14-2,8-2,2 5 9,NaN,14-6,4-1,7-0,5-0,2019/2019-04-08.pdf
4,Michigan St.,5,71,6,32-7,20-4,12-3,8-4,NaN,31,15-5,7-2,6-0,4-0,2019/2019-04-08.pdf
